## Here we reduce and concatenate the stats

In [1]:
import pandas as pd
import numpy as np
import os
import json
from tqdm import tqdm

def get_fixed_column_names():
    """
    Generate a fixed list of column names to ensure consistency across all files
    """
    columns = []
    
    # Models (excluding model_5)
    models = [f"model_{i}" for i in range(1, 7) if i != 5]
    
    # Metrics for each model
    metrics = ['gold_lp', 'rank', 'Entropy', 'gold_lp_cumsum', 'gold_prob', 'gold_lp_normalized']
    
    # Statistics to compute
    stats = ['median', 'max', 'min', 'std', 
             'moment1', 'moment2', 'moment3', 'moment4', 'moment5', 
             'q95', 'q90', 'q85', 'q80', 'q20', 'q15', 'q10', 'q05']
    
    # Generate columns for model metrics
    for model in models:
        for metric in metrics:
            for stat in stats:
                columns.append(f"{model}_{metric}_{stat}")
    
    # Generate columns for KL divergence terms
    model_pairs = [(i, j) for i in range(1, 7) for j in range(i+1, 7) 
                  if i != 5 and j != 5]
    
    for i, j in model_pairs:
        for stat in stats:
            columns.append(f"kl_{i}_vs_{j}_{stat}")
    
    return columns

def get_label_from_json(trial_name, question_number, label_folder_path):
    """
    Get label from corresponding JSON file and return 'fact' or 'hallucination'
    based on the 'statement' field
    
    Args:
        trial_name: Name of the trial (e.g., 'NCT001')
        question_number: Question number (1-7)
        label_folder_path: Path to the folder containing JSON files
    
    Returns:
        str: 'fact' if statement is True, 'hallucination' if False, None if error
    """
    if not label_folder_path:  # If no path provided
        return None
        
    json_path = os.path.join(
        label_folder_path,
        f"{trial_name}.json"
    )
    
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)
            
            # Get statement value
            statement = data.get('label')
            
            # Return appropriate label
            if statement == question_number:
                return 'fact'
            else:
                return 'hallucination'
                
    except FileNotFoundError:
        print(f"File not found: {json_path}")
        return None
    except json.JSONDecodeError:
        print(f"Invalid JSON in file: {json_path}")
        return None
    except Exception as e:
        print(f"Error reading JSON for {trial_name}: {e}")
        return None

def extract_trial_and_question(file_path):
    """
    Extract trial name and question number from file path
    Example: path/to/NCT00001959/logprob_matrix_1.csv -> ("NCT00001959", 1)
    """
    try:
        # Get filename and remove extension
        filename = os.path.basename(file_path)
        # Extract question number
        question_num = int(filename.split('_')[-1].split('.')[0])
        # Extract trial name from path
        trial_name = file_path.split('/')[-2]  # Adjust this based on your actual path structure
        return trial_name, question_num
    except Exception as e:
        print(f"Error extracting trial and question from {file_path}: {e}")
        return None, None

def compute_statistics(series):
    """
    Compute various statistical measures for a series
    """
    mean = series.mean()
    
    stats = {
        'median': series.median(),
        'max': series.max(),
        'min': series.min(),
        'std': series.std(),
        
        # Central moments
        'moment1': mean,
        'moment2': ((series - mean) ** 2).mean(),
        'moment3': ((series - mean) ** 3).mean(),
        'moment4': ((series - mean) ** 4).mean(),
        'moment5': ((series - mean) ** 5).mean(),
        
        # Existing quantiles
        'q95': series.quantile(0.95),
        'q90': series.quantile(0.90),
        'q85': series.quantile(0.85),
        'q80': series.quantile(0.80),
        'q20': series.quantile(0.20),
        'q15': series.quantile(0.15),
        'q10': series.quantile(0.10),
        'q05': series.quantile(0.05),
    }    
    return stats

def process_metrics_and_kl(input_folder, kl_folder, output_folder, label_folder_path=None):
    """
    Process original metrics and KL divergence terms with consistent column ordering
    """
    # Get fixed column names
    fixed_columns = get_fixed_column_names()
    
    # Original metrics to analyze
    metrics = ['gold_lp', 'rank', 'Entropy', 'gold_lp_cumsum', 'gold_prob', 'gold_lp_normalized']
    
    # Get all CSV files from model1 directory
    csv_files = []
    model1_path = os.path.join(input_folder, "model_1")
    for root, dirs, files in os.walk(model1_path):
        # Skip hidden directories
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        
        for file in files:
            if not file.startswith('.') and file.endswith('.csv'):
                csv_files.append((os.path.join(root, file), file))
    
    print(f"Found {len(csv_files)} CSV files to process")
    print(f"Output will have {len(fixed_columns) + 1} columns")  # +1 for label
    
    for file_path1, file_name in tqdm(csv_files, desc="Processing files"):
        try:
            # Get trial name and question number
            trial_name, question_num = extract_trial_and_question(file_path1)
            
            if trial_name and question_num and label_folder_path:
                # Get label from JSON only if path is provided
                label = get_label_from_json(trial_name, question_num, label_folder_path)
            else:
                label = None
            
            # Dictionary to store all metrics
            all_stats = {col: np.nan for col in fixed_columns}  # Initialize with NaN
            
            # Add label column
            all_stats['label'] = label
            
            # Process each model (excluding model_5)
            models = [f"model_{i}" for i in range(1, 7) if i != 5]
            
            # Step 1: Process original metrics for each model
            for model in models:
                file_path = os.path.join(input_folder, model, os.path.relpath(file_path1, model1_path))
                if not os.path.exists(file_path):
                    print(f"Skipping {file_name} - no matching file in {model}")
                    continue
                
                df = pd.read_csv(file_path)
                
                # Compute statistics for each metric
                for metric in metrics:
                    if metric in df.columns:
                        stats = compute_statistics(df[metric])
                        for stat_name, value in stats.items():
                            col_name = f"{model}_{metric}_{stat_name}"
                            all_stats[col_name] = value
            
            # Step 2: Process KL divergence terms
            kl_file_path = os.path.join(kl_folder, os.path.relpath(file_path1, model1_path))
            if os.path.exists(kl_file_path):
                kl_df = pd.read_csv(kl_file_path)
                
                # Get all KL columns
                kl_cols = [col for col in kl_df.columns if col.startswith('kl_')]
                
                # Compute statistics for each KL term
                for kl_col in kl_cols:
                    stats = compute_statistics(kl_df[kl_col])
                    for stat_name, value in stats.items():
                        col_name = f"{kl_col}_{stat_name}"
                        all_stats[col_name] = value
            
            # Create output DataFrame with fixed column order plus label
            columns_with_label = fixed_columns + ['label']
            result_df = pd.DataFrame([all_stats])[columns_with_label]
            
            # Create output directory structure
            output_file_path = os.path.join(output_folder, os.path.relpath(file_path1, model1_path))
            os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
            
            # Save results
            result_df.to_csv(output_file_path, index=False)
            
        except Exception as e:
            print(f"\nError processing {file_name}:")
            print(f"Error type: {type(e)}")
            print(f"Error message: {str(e)}")
            continue

def concatenate_output_files(output_folder):
    """
    Concatenate all CSV files in Output_folder and its subfolders into a single DataFrame
    """
    # Get list of all CSV files
    csv_files = []
    for root, dirs, files in os.walk(output_folder):
        # Skip hidden directories
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        
        for file in files:
            if not file.startswith('.') and file.endswith('.csv'):
                csv_files.append((os.path.join(root, file), file))
    
    print(f"Found {len(csv_files)} CSV files to concatenate")
    # Sort files alphabetically
    csv_files = sorted(csv_files)
    
    # Read and concatenate all files
    all_dfs = []
    for file_path, file_name in tqdm(csv_files, desc="Reading files"):
        try:
            df = pd.read_csv(file_path)
            
            # Add filename as a column (optional but useful for tracking)
            df['source_file'] = file_name
            
            # Add full path as a column (optional)
            df['file_path'] = os.path.relpath(file_path, output_folder)
            
            all_dfs.append(df)
            
        except Exception as e:
            print(f"\nError reading {file_name}:")
            print(f"Error type: {type(e)}")
            print(f"Error message: {str(e)}")
            continue
    
    # Concatenate all DataFrames
    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
        
        # Print some information about the final DataFrame
        print("\nFinal DataFrame info:")
        print(f"Shape: {final_df.shape}")
        print(f"Number of samples: {len(final_df)}")
        print(f"Number of features: {len(final_df.columns)}")
        
        if 'label' in final_df.columns:
            print("\nLabel distribution:")
            print(final_df['label'].value_counts())
        
        return final_df
    else:
        print("No files were successfully read!")
        return None

if __name__ == "__main__":
    # Paths
    input_folder = "Database_free_evaluation/MedQA/1-Run_ensemble/Features/Test"
    kl_folder = "kl_analysis_Paragraph_title_features"
    output_folder = "Output_folder_Paragraph_title"

     # Label folder path (set to None if not using labels)
    label_folder_path = "Database_free_evaluation/MedQA/1-Run_ensemble/Features/Test"
    # Or for other folders:
    # label_folder_path = None
    
    # Create output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Step 1: Process files and create individual outputs
    process_metrics_and_kl(input_folder, kl_folder, output_folder, label_folder_path)

    # Step 2: Concatenate all output files
    final_df = concatenate_output_files(output_folder)
    
    # Optional: Save concatenated DataFrame
    if final_df is not None:
        final_df.to_csv("concatenated_results_Paragraph_title_MEDQA_TESTSET.csv", index=False)

Found 5092 CSV files to process
Output will have 681 columns


Processing files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5092/5092 [08:57<00:00,  9.48it/s]


Found 5092 CSV files to concatenate


Reading files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5092/5092 [00:37<00:00, 135.24it/s]



Final DataFrame info:
Shape: (5092, 683)
Number of samples: 5092
Number of features: 683

Label distribution:
label
hallucination    3819
fact             1273
Name: count, dtype: int64


In [2]:
final_df.head(20)

,model_1_gold_lp_median,model_1_gold_lp_max,model_1_gold_lp_min,model_1_gold_lp_std,model_1_gold_lp_moment1,model_1_gold_lp_moment2,model_1_gold_lp_moment3,model_1_gold_lp_moment4,model_1_gold_lp_moment5,model_1_gold_lp_q95,...,kl_4_vs_6_q90,kl_4_vs_6_q85,kl_4_vs_6_q80,kl_4_vs_6_q20,kl_4_vs_6_q15,kl_4_vs_6_q10,kl_4_vs_6_q05,label,source_file,file_path
0,-0.000019,0.000000e+00,-6.800458,1.697467,-0.437432,2.701308,-16.027784,102.487482,-651.912252,0.000000e+00,...,0.005622,0.001029,0.000649,1.595385e-06,7.912583e-07,4.297463e-07,2.674032e-07,hallucination,logprob_matrix_1.csv,NCT00001/logprob_matrix_1.csv
1,-0.000060,-3.099437e-06,-9.929261,3.066088,-1.226612,8.677751,-56.598351,476.950235,-3992.945736,-3.886215e-06,...,0.007044,0.002628,0.001053,1.854693e-05,1.575596e-05,1.293821e-05,1.159757e-05,fact,logprob_matrix_2.csv,NCT00001/logprob_matrix_2.csv
2,-0.000012,-4.887569e-06,-8.588479,2.979762,-1.379453,7.892427,-41.170908,306.655102,-2168.715217,-5.793554e-06,...,0.176756,0.022030,0.011593,1.739765e-05,1.662805e-05,1.400521e-05,9.529106e-06,hallucination,logprob_matrix_3.csv,NCT00001/logprob_matrix_3.csv
3,-0.000231,-2.503392e-06,-13.147138,4.580456,-2.144122,18.649398,-147.036681,1669.845832,-17983.540422,-3.790847e-06,...,0.037156,0.008888,0.005415,1.154381e-05,9.138916e-06,6.678054e-06,4.161223e-06,hallucination,logprob_matrix_4.csv,NCT00001/logprob_matrix_4.csv
4,-0.024554,-1.430511e-06,-9.694468,3.954765,-2.151101,13.685148,-63.124516,503.658168,-3449.549530,-1.336314e-05,...,0.144194,0.122152,0.112784,2.738656e-05,1.731055e-05,1.234869e-05,8.239181e-06,hallucination,logprob_matrix_1.csv,NCT00002/logprob_matrix_1.csv
5,-0.000045,0.000000e+00,-9.694468,3.521934,-1.576554,11.276382,-65.103798,517.527563,-3903.710350,-1.907345e-06,...,0.167387,0.084115,0.000844,8.359097e-06,5.931673e-06,3.504249e-06,1.771608e-06,hallucination,logprob_matrix_2.csv,NCT00002/logprob_matrix_2.csv
6,-0.004601,-1.072883e-06,-7.741167,3.173168,-1.834848,8.390828,-30.718271,211.026368,-1185.132288,-1.400708e-06,...,0.363050,0.277777,0.192503,1.787255e-06,1.614053e-06,1.440851e-06,1.267649e-06,hallucination,logprob_matrix_3.csv,NCT00002/logprob_matrix_3.csv
7,-0.001007,-1.645075e-05,-9.937939,3.752193,-1.428837,12.067675,-85.568573,752.392199,-6367.735112,-1.684413e-05,...,0.169209,0.157562,0.123565,5.311033e-06,3.429367e-06,2.779497e-06,2.129628e-06,fact,logprob_matrix_4.csv,NCT00002/logprob_matrix_4.csv
8,-0.000703,0.000000e+00,-10.000000,4.038449,-2.218232,14.270434,-67.816366,558.303077,-3956.081289,0.000000e+00,...,0.115345,0.017098,0.011064,1.021602e-06,1.890836e-07,5.418659e-08,3.555976e-08,hallucination,logprob_matrix_1.csv,NCT00003/logprob_matrix_1.csv
9,-0.001928,0.000000e+00,-9.479224,3.562985,-1.403883,10.881311,-73.090317,610.436048,-4901.707326,-1.133659e-05,...,0.267148,0.067762,0.001110,3.606053e-05,2.927973e-05,1.973740e-05,1.019507e-05,fact,logprob_matrix_2.csv,NCT00003/logprob_matrix_2.csv
